In [ ]:
import numpy as np 
from scipy.interpolate import RegularGridInterpolator
from scipy.integrate import solve_ivp

def integrate_particles(positions, concentrations, velocity_field, X, Y, dt, num_steps, L):
    #positions is of the form np.ones((6, 2)), concentrations of the form np.ones((6, 1))
    num_particles = len(positions)
    positions_over_time = np.zeros((num_steps+1, num_particles, 2))  #positions_over_time[0] gives positions at t=0
    concentrations_over_time = np.zeros((num_steps+1, num_particles, 1))
    positions_over_time[0] = positions
    concentrations_over_time[0] = concentrations

    u, v = get_matrix_u(velocity_field), get_matrix_v(velocity_field)
    polynomial_u, polynomial_v = interpolate(u), interpolate(v)

    t_span = (0, num_steps * dt)
    t_eval = [i * dt for i in range(num_steps+1)]
    #y0 = [ [positions[i][0], positions[i][1]] for i in range(num_particles)]
    t = [i*dt for i in range(num_steps+1)]
    y0 = positions
    vector_rk4(fun, y0, t, polynomial_u, polynomial_v)
    #concentration starts here
    du, dv = get_matrix_du(u), get_matrix_dv(v)
    polynomial_du, polynomial_dv = interpolate(du), interpolate(dv)
    y0conc = concentrations_over_time
    vector_rk4_conc(funconc, y0conc, t, polynomial_du, polynomial_dv)

    return positions_over_time, concentrations_over_time


def get_matrix_u(velocity_field):
    umatrix = velocity_field
    nt, nx, ny = velocity_field.shape
    for i in range(nt):
        umatrix[i] = np.gradient(velocity_field[i], axis=1) * -1
    return umatrix

def get_matrix_v(velocity_field):
    vmatrix = velocity_field
    nt, nx, ny = velocity_field.shape
    for i in range(nt):
        vmatrix[i] = np.gradient(velocity_field[i], axis=0) 
    return vmatrix

def get_matrix_du(umatrix):
    dumatrix = umatrix
    nt, nx, ny = umatrix.shape
    for i in range(nt):
        dumatrix[i] = np.gradient(umatrix[i], axis=0) 
    return dumatrix

def get_matrix_dv(umatrix):
    dumatrix = umatrix
    nt, nx, ny = umatrix.shape
    for i in range(nt):
        dumatrix[i] = np.gradient(umatrix[i], axis=1) 
    return dumatrix

def interpolate(umatrix):

    t = umatrix.shape[0]
    time = [i for i in range(t)]

    interpolator = RegularGridInterpolator((time, X, Y), umatrix, method = "cubic")

    return interpolator

def funconc(t, y, polynomial_du, polynomial_dv):
    #concentrations of the form np.ones((6, 1))
    #-c (du/dx + dv/dy) = result
    c = y
    xcoords = positions_over_time[t][:, 0]
    xcoords = np.array(xcoords).reshape(-1, 1)
    ycoords = positions_over_time[t][:, 1]
    ycoords = np.array(xcoords).reshape(-1, 1)
    timecoords = t*np.ones(num_particles)
    coords = [[x, y, z] for x, y, z in zip(timecoords, xlist, ylist)]

    dudx = polynomial_du(coords)
    dvdy = polynomial_dv(coords)
    result = -1*c*(dudx + dvdy)
    return result


def vector_rk4(f, y0, t, polynomial_u, polynomial_v):
    #y0 will be of shape num_particles(N) x 2, same for all k values 
    #t = [dt, 2dt, 3dt,...]
    #f takes a scalar t[i] and a N x 2 array
    n = len(t)
    #y = np.zeros((n, len(y0))) #this is positions_over_time
    y = positions_over_time
    #y[0] = y0 #we have already set the firstvalue
    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = f(t[i], y[i], polynomial_u, polynomial_v)
        k2 = f(t[i] + 0.5*h, y[i] + 0.5*h*k1, polynomial_u, polynomial_v)
        k3 = f(t[i] + 0.5*h, y[i] + 0.5*h*k2, polynomial_u, polynomial_v)
        k4 = f(t[i+1], y[i] + h*k3, polynomial_u, polynomial_v)
        y[i+1] = y[i] + (h/6)*(k1 + 2*k2 + 2*k3 + k4)
    return y

def vector_rk4_conc(f, y0, t, polynomial_du, polynomial_dv):
    #y0 will be of shape num_particles(N) x 2, same for all k values 
    #t = [dt, 2dt, 3dt,...]
    #f takes a scalar t[i] and a N x 2 array
    n = len(t)
    #y = np.zeros((n, len(y0))) #this is positions_over_time
    y = concentrations_over_time
    #y[0] = y0 #we have already set the firstvalue
    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = f(t[i], y[i], polynomial_du, polynomial_dv)
        k2 = f(t[i] + 0.5*h, y[i] + 0.5*h*k1, polynomial_du, polynomial_dv)
        k3 = f(t[i] + 0.5*h, y[i] + 0.5*h*k2, polynomial_du, polynomial_dv)
        k4 = f(t[i+1], y[i] + h*k3, polynomial_du, polynomial_dv)
        y[i+1] = y[i] + (h/6)*(k1 + 2*k2 + 2*k3 + k4)
    return y

def fun(t, y, polynomial_u, polynomial_v):
    #note that here y refers to both x and y coords
    #print(y)
    #print(2*L)
    #print([(sublist[0]+L)% (2*L) -L for sublist in y] )

    xlist = [(sublist[0]+L)% (2*L) -L for sublist in y] 
    ylist = [(sublist[1]+L)% (2*L) -L for sublist in y] 
    #print(np.array(xlist).reshape(-1, 1))
    #print(t*np.ones(num_particles))
    timecoords = t*np.ones(num_particles)

    #coords = [timecoords, xlist, ylist]
    coords = [[x, y, z] for x, y, z in zip(timecoords, xlist, ylist)]
    #print(coords)
    #a=[[0,1,2],[0,1,2],[0,1,2], [0,1,2]]
    #print(a)
    #print(polynomial_u(a))
    #print(coords)
    #print(polynomial_u(coords))
    dxdt = polynomial_u(coords) 
    dydt = polynomial_v(coords)
    dxdt=np.array(dxdt).reshape(-1, 1)
    dydt=np.array(dydt).reshape(-1, 1)
 

    result = np.concatenate((dxdt, dydt), axis=1)


    return result

In [ ]:
import numpy as np 
from scipy.interpolate import RegularGridInterpolator
from scipy.integrate import solve_ivp

def integrate_particles(positions, concentrations, velocity_field, X, Y, dt, num_steps, L):
    #positions is of the form np.ones((6, 2)), concentrations of the form np.ones((6, 1))
    num_particles = len(positions)
    positions_over_time = np.zeros((num_steps+1, num_particles, 2))  #positions_over_time[0] gives positions at t=0, right shape checked!
    concentrations_over_time = np.zeros((num_steps+1, num_particles, 1))
    positions_over_time[0] = positions
    #print(positions_over_time) still correct shape!
    concentrations_over_time[0] = concentrations

    u, v = get_matrix_u(velocity_field), get_matrix_v(velocity_field)
    polynomial_u, polynomial_v = interpolate(u), interpolate(v)
    #print(positions_over_time) still correct shape!
    t_span = (0, num_steps * dt)
    t_eval = [i * dt for i in range(num_steps+1)]
    #y0 = [ [positions[i][0], positions[i][1]] for i in range(num_particles)]
    t = [i*dt for i in range(num_steps+1)]
    y0 = positions
    #print(positions_over_time) still correct shape
    vector_rk4(fun, y0, t, polynomial_u, polynomial_v, positions_over_time)
    #concentration starts here
    du, dv = get_matrix_du(u), get_matrix_dv(v)
    polynomial_du, polynomial_dv = interpolate(du), interpolate(dv)
    vector_rk4_conc(funconc, concentrations_over_time, t, polynomial_du, polynomial_dv)

    return positions_over_time, concentrations_over_time


def get_matrix_u(velocity_field):
    umatrix = velocity_field
    nt, nx, ny = velocity_field.shape
    for i in range(nt):
        umatrix[i] = np.gradient(velocity_field[i], axis=1) * -1
    return umatrix

def get_matrix_v(velocity_field):
    vmatrix = velocity_field
    nt, nx, ny = velocity_field.shape
    for i in range(nt):
        vmatrix[i] = np.gradient(velocity_field[i], axis=0) 
    return vmatrix

def get_matrix_du(umatrix):
    dumatrix = umatrix
    nt, nx, ny = umatrix.shape
    for i in range(nt):
        dumatrix[i] = np.gradient(umatrix[i], axis=0) 
    return dumatrix

def get_matrix_dv(umatrix):
    dumatrix = umatrix
    nt, nx, ny = umatrix.shape
    for i in range(nt):
        dumatrix[i] = np.gradient(umatrix[i], axis=1) 
    return dumatrix

def interpolate(umatrix):

    t = umatrix.shape[0]
    time = [i for i in range(t)]

    interpolator = RegularGridInterpolator((time, X, Y), umatrix, method = "cubic")

    return interpolator

def vector_rk4(f, y0, t, polynomial_u, polynomial_v, positions_over_time):
    #y0 will be of shape num_particles(N) x 2, same for all k values 
    #t = [dt, 2dt, 3dt,...]
    #f takes a scalar t[i] and a N x 2 array
    n = len(t)
    #y = np.zeros((n, len(y0))) #this is positions_over_time
    y = positions_over_time
    #print(y)
    #print("positions above should be 5")
    #y[0] = y0 #we have already set the firstvalue

    #testing
    #print(t)
    #print(n)
    #print(y)
    #testing

    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = f(t[i], y[i], polynomial_u, polynomial_v)
        k2 = f(t[i] + 0.5*h, y[i] + 0.5*h*k1, polynomial_u, polynomial_v)
        k3 = f(t[i] + 0.5*h, y[i] + 0.5*h*k2, polynomial_u, polynomial_v)
        k4 = f(t[i+1], y[i] + h*k3, polynomial_u, polynomial_v)
        y[i+1] = y[i] + (h/6)*(k1 + 2*k2 + 2*k3 + k4)
    return y

def fun(t, y, polynomial_u, polynomial_v):
    #note that here y refers to both x and y coords
    #print(y)
    #print(2*L)
    #print([(sublist[0]+L)% (2*L) -L for sublist in y] )
    #print(y)
    #print(" y above")
    xlist = [(sublist[0]+L)% (2*L) -L for sublist in y] 
    ylist = [(sublist[1]+L)% (2*L) -L for sublist in y] 
    #print(np.array(xlist).reshape(-1, 1))
    #print(t*np.ones(num_particles))
    timecoords = t*np.ones(num_particles)
    #print(xlist)
    #coords = [timecoords, xlist, ylist]
    coords = [[x, y, z] for x, y, z in zip(timecoords, xlist, ylist)]
    #print(coords)
    #a=[[0,1,2],[0,1,2],[0,1,2], [0,1,2]]
    #print(a)
    #print(polynomial_u(a))
    #print(coords)
    #print(polynomial_u(coords))
    dxdt = polynomial_u(coords) 
    dydt = polynomial_v(coords)
    dxdt=np.array(dxdt).reshape(-1, 1)
    dydt=np.array(dydt).reshape(-1, 1)
 

    result = np.concatenate((dxdt, dydt), axis=1)
    #print(result)

    return result

def vector_rk4_conc(f, y, t, polynomial_du, polynomial_dv):
    n = len(t)
    #print(y)
    #print("concentrations over time above should be 4+1") fixed
    #y[0] = y0 #we have already set the firstvalue
    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = f(t[i], y[i], polynomial_du, polynomial_dv)
        #testing
        print(t[i] + 0.5*h, y[i] + 0.5*h*k1)
        #testing
        k2 = f(t[i] + 0.5*h, y[i] + 0.5*h*k1, polynomial_du, polynomial_dv)
        k3 = f(t[i] + 0.5*h, y[i] + 0.5*h*k2, polynomial_du, polynomial_dv)
        k4 = f(t[i+1], y[i] + h*k3, polynomial_du, polynomial_dv)
        y[i+1] = y[i] + (h/6)*(k1 + 2*k2 + 2*k3 + k4)
    return y

def funconc(t, y, polynomial_du, polynomial_dv):
    #concentrations of the form np.ones((6, 1))
    #-c (du/dx + dv/dy) = result
    #print(y)
    c = y
    #print(positions_over_time[t])#t = 0.5 at that time so need something else of course!
    #xcoords = positions_over_time[t][:, 0]
    #print(xcoords) 
    #xcoords = np.array(xcoords).reshape(-1, 1)
    #ycoords = positions_over_time[t][:, 1]
    #ycoords = np.array(xcoords).reshape(-1, 1)
    #get x and y coords at time t
    timecoords = t*np.ones(num_particles)
    #print(timecoords)
    coords = [[x, y, z] for x, y, z in zip(timecoords, xcoords, ycoords)]
    #print(c)
    #print("c is above") works until here
    dudx = polynomial_du(coords)
    #print(dudx)
    #print("dudx above")
    dvdy = polynomial_dv(coords)
    #print(dvdy)
    #print(dvdy+dudx)
    column=np.array(dvdy+dudx).reshape(-1, 1)
    #print(column)
    result = -1*c*(column)
    #print(result)
    return result